In [1]:
# 2. Importar bibliotecas
import os
import time
import cupy as cp
import numpy as np # Ainda precisamos do numpy para operações de CPU com Pandas
import pandas as pd
from cupyx.scipy.ndimage import binary_dilation

In [2]:
# 3. Definir caminhos e estrutura
base_dir = r'E:\Process'

# Verificar se o diretório existe
if not os.path.exists(base_dir):
    print(f"O diretório {base_dir} não existe.")
else:
    print(f"Diretório definido: {base_dir}")

# Structure for binary dilation - stays on GPU
structure = cp.array([[0, 1, 0],
                      [1, 1, 1],
                      [0, 1, 0]], dtype=cp.uint8)

Diretório definido: E:\Process


In [6]:


# 4. Funções de processamento otimizadas
def analisar_vizinhança_gpu_vetorizado(patches_A_gpu, patches_B_gpu):
    print("Iniciando análise de vizinhança na GPU (vetorizado)...")
    start_time = time.time()

    # Obter IDs de patches únicos em patches_A, ignorando 0
    patch_ids_A = cp.unique(patches_A_gpu[patches_A_gpu > 0])
    total_patches = len(patch_ids_A)
    print(f"Total de patches únicos a analisar: {total_patches}")

    # Inicializar arrays para armazenar resultados na GPU
    results_patch_id = cp.empty(total_patches, dtype=cp.int32)
    results_vizinhanza = cp.empty(total_patches, dtype=cp.uint8)
   
    max_patch_id_A = int(patch_ids_A.max()) # Converte para int para evitar problemas com índices

    # Mapa para armazenar os resultados
    vizinhanza_map = cp.zeros(max_patch_id_A + 1, dtype=cp.uint8)

    # Processar em blocos para gerenciar memória GPU e progresso
    BLOCO_TAMANHO_GPU = 1000 # Ajuste este valor conforme a memória da GPU e o número de patches

    for i in range(0, total_patches, BLOCO_TAMANHO_GPU):
        bloco_ids = patch_ids_A[i : min(i + BLOCO_TAMANHO_GPU, total_patches)]
        print(f"  Processando bloco de patches {i+1} a {min(i + BLOCO_TAMANHO_GPU, total_patches)} de {total_patches} na GPU...")

    all_patch_ids_A = cp.unique(patches_A_gpu[patches_A_gpu > 0])
    total_unique_patches = len(all_patch_ids_A)

    # Prepare arrays para resultados, já na GPU
    results_patch_ids = cp.empty(total_unique_patches, dtype=cp.int32)
    results_vizinhanza = cp.empty(total_unique_patches, dtype=cp.uint8)

       # Este é o principal ponto de otimização para seu caso.

    for i, patch_id_val in enumerate(all_patch_ids_A):
        # As operações abaixo são todas CuPy e rodam na GPU
        mask_A = patches_A_gpu == patch_id_val
        dilated = binary_dilation(mask_A, structure=structure)
        border = cp.logical_and(dilated, ~mask_A)

        neighbor_values = patches_B_gpu[border]
        neighbor_ids = cp.unique(neighbor_values[neighbor_values > 0])

        # Lógica de vizinhança (ainda serializada por patch_id, mas operando rápido na GPU)
        viz = 0 # Inicializa com um valor padrão, caso nenhuma condição seja atendida
        if len(neighbor_ids) == 0:
            viz = 1
        elif len(neighbor_ids) == 1:
            # Atenção: cp.any e cp.all em CuPy operam em arrays CuPy.
            # Se neighbor_values for um array CuPy, cp.any/cp.all funcionará.
            if cp.any(neighbor_values == 0): # Verifica se há algum pixel "estável" (valor 0) na vizinhança
                viz = 2
            elif cp.all(neighbor_values == neighbor_ids[0]): # Verifica se todos os pixels de vizinhos são o mesmo ID
                viz = 4
            else: # Diferentes IDs na vizinhança, mas apenas um único neighbor_id (ex: 0 e um outro ID)
                viz = 2
        else: # Mais de um neighbor_id
            viz = 3

        results_patch_ids[i] = patch_id_val
        results_vizinhanza[i] = viz

        if (i + 1) % 1000 == 0: # Feedback de progresso
            print(f"  Processados {i+1} de {total_unique_patches} patches...")
            cp.cuda.Stream.null.synchronize() # Garante que as operações da GPU sejam concluídas para um tempo preciso

    # Transferir os resultados finais da GPU para a CPU APENAS UMA VEZ
    final_patch_ids = cp.asnumpy(results_patch_ids)
    final_vizinhanza = cp.asnumpy(results_vizinhanza)

    print(f"Análise de vizinhança concluída em {time.time() - start_time:.2f} segundos.")
    return list(zip(final_patch_ids, final_vizinhanza))


# 5. Verificar GPU (permanece o mesmo)
print("GPU disponível:", cp.cuda.Device(0).attributes.get('name', 'Nome não disponível'))
cp.cuda.Device(0).use() # Garante que a GPU padrão seja usada

# 6. Carregar arquivos .npy (permanece o mesmo, carregam diretamente para GPU)
print("Carregando arquivos .npy para a GPU...")
ganho = cp.load(os.path.join(base_dir, 'ganho_patches_2015_2016.npy'))
perda = cp.load(os.path.join(base_dir, 'perda_patches_2015_2016.npy'))
estavel = cp.load(os.path.join(base_dir, 'estavel_patches_2015_2016.npy'))
print("Arquivos .npy carregados.")

# 7. Executar análise com a função otimizada
print("\nAnalisando perda...")
resultado_perda = analisar_vizinhança_gpu_vetorizado(perda, estavel)

print("\nAnalisando ganho...")
resultado_ganho = analisar_vizinhança_gpu_vetorizado(ganho, estavel)


# 8. Salvar resultados em CSV (permanece o mesmo)
print("\nSalvando resultados em CSV...")
df_perda = pd.DataFrame(resultado_perda, columns=['patch_id', 'vizinhança'])
df_ganho = pd.DataFrame(resultado_ganho, columns=['patch_id', 'vizinhança'])

# Verifique se o nome do arquivo de saída está correto (era 1985_1986, agora 1986_1987)
df_perda.to_csv(os.path.join(base_dir, 'resultado_vizinhanca_perda_2015_2016.csv'), index=False)
df_ganho.to_csv(os.path.join(base_dir, 'resultado_vizinhanca_ganho_2015_2016.csv'), index=False)

print("Resultados salvos com sucesso.")

GPU disponível: Nome não disponível
Carregando arquivos .npy para a GPU...
Arquivos .npy carregados.

Analisando perda...
Iniciando análise de vizinhança na GPU (vetorizado)...
Total de patches únicos a analisar: 214972
  Processando bloco de patches 1 a 1000 de 214972 na GPU...
  Processando bloco de patches 1001 a 2000 de 214972 na GPU...
  Processando bloco de patches 2001 a 3000 de 214972 na GPU...
  Processando bloco de patches 3001 a 4000 de 214972 na GPU...
  Processando bloco de patches 4001 a 5000 de 214972 na GPU...
  Processando bloco de patches 5001 a 6000 de 214972 na GPU...
  Processando bloco de patches 6001 a 7000 de 214972 na GPU...
  Processando bloco de patches 7001 a 8000 de 214972 na GPU...
  Processando bloco de patches 8001 a 9000 de 214972 na GPU...
  Processando bloco de patches 9001 a 10000 de 214972 na GPU...
  Processando bloco de patches 10001 a 11000 de 214972 na GPU...
  Processando bloco de patches 11001 a 12000 de 214972 na GPU...
  Processando bloco de

In [4]:
!pip install rasterio

In [7]:
import os
import numpy as np
import pandas as pd
import rasterio
from rasterio.transform import from_origin
from rasterio.enums import Compression
from rasterio.errors import RasterioIOError # Importar para tratar erros de I/O

# Diretório base no Google Drive
base_dir = r'E:\Process'

# Parâmetros do raster
# Consistência nos nomes das variáveis (camelCase para snake_case, por convenção Python)
COLS = 12989
ROWS = 9278
CELL_SIZE_X = 0.0002694945852358565
CELL_SIZE_Y = 0.00026949458523585663
UPPER_LEFT_X = -55.500254
UPPER_LEFT_Y = -16.499806

# Transformação geográfica
transform = from_origin(UPPER_LEFT_X, UPPER_LEFT_Y, CELL_SIZE_X, CELL_SIZE_Y)

# Sistema de coordenadas WGS 84 (EPSG:4326)
CRS = 'EPSG:4326'

# Função otimizada para converter CSV de vizinhança em GeoTIFF usando referência de patches
def vizinhanca_para_geotiff_otimizado(csv_path, npy_patch_path, output_tif_path):
    print(f"Iniciando conversão: {os.path.basename(csv_path)}")

    try:
        # Carregar CSV: usecols para carregar apenas as colunas necessárias, reduzindo o consumo de memória
        df = pd.read_csv(csv_path, usecols=['patch_id', 'vizinhança'])
    except FileNotFoundError:
        print(f"Erro: Arquivo CSV não encontrado em {csv_path}")
        return
    except pd.errors.EmptyDataError:
        print(f"Erro: Arquivo CSV vazio ou mal formatado em {csv_path}")
        return

    try:
        # Carregar patches
        patches = np.load(npy_patch_path)
    except FileNotFoundError:
        print(f"Erro: Arquivo NPY não encontrado em {npy_patch_path}")
        return
    except Exception as e:
        print(f"Erro ao carregar arquivo NPY {npy_patch_path}: {e}")
        return

    # Garantir que as dimensões de patches correspondam às esperadas
    if patches.shape != (ROWS, COLS):
        print(f"Atenção: Dimensões do arquivo NPY ({patches.shape}) não correspondem às esperadas ({ROWS}, {COLS}).")
        # Você pode optar por redimensionar ou levantar um erro dependendo do comportamento desejado
        # Ex: patches = np.resize(patches, (ROWS, COLS))

    # Inicializar o raster de saída com zeros, usando o mesmo dtype dos valores de vizinhança ou np.uint8
    # Se 'vizinhança' puder ter valores fora de uint8, ajuste o dtype (ex: np.int16)
    # É crucial que raster_out tenha o mesmo tipo de dados do output GeoTIFF para eficiência
    raster_out = np.zeros_like(patches, dtype=np.uint8) # Assumindo que 'vizinhança' cabe em uint8

    # Otimização principal: Mapeamento vetorizado
    # 1. Criar um array auxiliar para mapear patch_id para vizinhança
    # Determine o valor máximo de patch_id para dimensionar o array de mapeamento
    max_patch_id = df['patch_id'].max()
    if max_patch_id >= (2**32 - 1): # Se patch_id for muito grande, usar um dicionário pode ser melhor
        print("Atenção: patch_id muito grande, usando dicionário para mapeamento.")
        patch_id_to_vizinhanza = dict(zip(df['patch_id'], df['vizinhança']))
        # Iterar sobre os patches e aplicar o mapeamento
        for i in np.unique(patches): # Itera sobre os IDs de patches únicos presentes no raster
            if i in patch_id_to_vizinhanza:
                raster_out[patches == i] = patch_id_to_vizinhanza[i]
    else:
        # Cria um array de lookup onde o índice é o patch_id e o valor é a vizinhança
        # Inicializa com 0 (ou um valor de 'no data' apropriado)
        lookup_table = np.zeros(max_patch_id + 1, dtype=np.uint8)
        # Preenche a lookup table com os valores de vizinhança
        lookup_table[df['patch_id'].values] = df['vizinhança'].values

        # Aplica o mapeamento vetorizado: para cada pixel em 'patches', use seu valor como índice na lookup_table
        # Isso é SIGNIFICATIVAMENTE mais rápido do que o loop original.
        raster_out = lookup_table[patches]

    print("Salvando GeoTIFF...")
    try:
        with rasterio.open(
            output_tif_path,
            'w',
            driver='GTiff',
            height=ROWS,
            width=COLS,
            count=1,
            dtype=raster_out.dtype, # Usa o dtype do array de saída
            crs=CRS,
            transform=transform,
            compress=Compression.lzw, # Compressão LZW é lossless e eficiente
            tiled=True,              # Ativa o tiling para melhor performance de leitura/escrita em arquivos grandes
            blockxsize=256,          # Tamanho do bloco para tiling (ajustar se necessário)
            blockysize=256           # Tamanho do bloco para tiling
        ) as dst:
            dst.write(raster_out, 1)
        print(f"GeoTIFF salvo: {output_tif_path}\n")
    except RasterioIOError as e:
        print(f"Erro de I/O ao salvar GeoTIFF {output_tif_path}: {e}")
    except Exception as e:
        print(f"Erro inesperado ao salvar GeoTIFF {output_tif_path}: {e}")

# Caminhos dos arquivos
csv_perda = os.path.join(base_dir, 'resultado_vizinhanca_perda_2015_2016.csv')
csv_ganho = os.path.join(base_dir, 'resultado_vizinhanca_ganho_2015_2016.csv')
npy_perda = os.path.join(base_dir, 'perda_patches_2015_2016.npy')
npy_ganho = os.path.join(base_dir, 'ganho_patches_2015_2016.npy')
tif_perda = os.path.join(base_dir, 'vizinhanca_perda_2015_2016.tif')
tif_ganho = os.path.join(base_dir, 'vizinhanca_ganho_2015_2016.tif')

# Executar conversão com a função otimizada
vizinhanca_para_geotiff_otimizado(csv_perda, npy_perda, tif_perda)
vizinhanca_para_geotiff_otimizado(csv_ganho, npy_ganho, tif_ganho)

Iniciando conversão: resultado_vizinhanca_perda_2015_2016.csv
Salvando GeoTIFF...
GeoTIFF salvo: E:\Process\vizinhanca_perda_2015_2016.tif

Iniciando conversão: resultado_vizinhanca_ganho_2015_2016.csv
Salvando GeoTIFF...
GeoTIFF salvo: E:\Process\vizinhanca_ganho_2015_2016.tif

